In [6]:
import numpy as np

In [29]:
X, Y = np.load('data/lab4.p', allow_pickle=True)
x_train, x_test = X
y_train, y_test = Y

In [24]:
x_train.shape

(2, 120)

In [32]:
y_train.shape

(2, 120)

In [33]:
x_test.shape

(2, 80)

In [36]:
y_test.shape

(2, 80)